In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('Reddit_Encoded.csv')
df

,comment,hate_speech,lemmatized_comment,document_vector_flat
0,subsection retarded hungarians ohh boy brace l...,1,subsection retard hungarians ohh boy brace liv...,"0.014043219,-0.01809359,0.017145459,0.08062436..."
1,hiii just got work Foundation and grounding ma...,0,hiii just get work Foundation and ground mainl...,"-0.0030388932,-0.035133556,0.020659983,0.07383..."
2,wow guess soyboys every country,0,wow guess soyboys every country,"0.017362628,0.005587179,0.0297773,0.109146975,..."
3,owen benjamins soyboy song goes every country ...,0,owen benjamins soyboy song go every country amaze,"0.018085241,0.0011954829,2.8959475e-05,0.07601..."
4,yall hear sumn means live small town rn for w...,0,yall hear sumn mean live small town rn for wor...,"0.023993038,-0.00060867134,0.005239945,0.05989..."
...,...,...,...,...
22206,op stop faggot post videos next time hard,1,op stop faggot post videos next time hard,"0.08339707,-0.017676119,-0.036872935,0.1038081..."
22207,minute long video top hate champagne goes need...,0,minute long video top hate champagne go need g...,"0.009342635,0.02728245,-0.0013451587,0.0765217..."
22208,clue whos ecelebs are point time need get alo...,1,clue whos ecelebs be point time need get along...,"-0.0006810841,0.008315975,0.029024707,0.077029..."
22209,didn’t insult you insult me,0,didn ’ t insult you insult me,"0.06474433,-0.17082486,-0.029483724,0.04382453..."


In [3]:
# Define the chunk size
chunk_size = 1000
file_path = "Reddit_Encoded.csv"

# Initialize an empty list to store the data
data_chunks = []

# Load data in chunks
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    data_chunks.append(chunk)

# Combine the chunks into a single DataFrame
df = pd.concat(data_chunks, ignore_index=True)

In [32]:
data_chunks

[                                               comment  hate_speech  \
 0    subsection retarded hungarians ohh boy brace l...            1   
 1    hiii just got work Foundation and grounding ma...            0   
 2                      wow guess soyboys every country            0   
 3    owen benjamins soyboy song goes every country ...            0   
 4     yall hear sumn means live small town rn for w...            0   
 ..                                                 ...          ...   
 995                                    cunt prison yet            1   
 996             honestly woman lets cum inside special            0   
 997              special unable score special retarded            1   
 998                                                yes            0   
 999                                               hehe            0   
 
                                     lemmatized_comment  \
 0    subsection retard hungarians ohh boy brace liv...   
 1    hiii just 

In [4]:
df

,comment,hate_speech,lemmatized_comment,document_vector_flat
0,subsection retarded hungarians ohh boy brace l...,1,subsection retard hungarians ohh boy brace liv...,"0.014043219,-0.01809359,0.017145459,0.08062436..."
1,hiii just got work Foundation and grounding ma...,0,hiii just get work Foundation and ground mainl...,"-0.0030388932,-0.035133556,0.020659983,0.07383..."
2,wow guess soyboys every country,0,wow guess soyboys every country,"0.017362628,0.005587179,0.0297773,0.109146975,..."
3,owen benjamins soyboy song goes every country ...,0,owen benjamins soyboy song go every country amaze,"0.018085241,0.0011954829,2.8959475e-05,0.07601..."
4,yall hear sumn means live small town rn for w...,0,yall hear sumn mean live small town rn for wor...,"0.023993038,-0.00060867134,0.005239945,0.05989..."
...,...,...,...,...
22206,op stop faggot post videos next time hard,1,op stop faggot post videos next time hard,"0.08339707,-0.017676119,-0.036872935,0.1038081..."
22207,minute long video top hate champagne goes need...,0,minute long video top hate champagne go need g...,"0.009342635,0.02728245,-0.0013451587,0.0765217..."
22208,clue whos ecelebs are point time need get alo...,1,clue whos ecelebs be point time need get along...,"-0.0006810841,0.008315975,0.029024707,0.077029..."
22209,didn’t insult you insult me,0,didn ’ t insult you insult me,"0.06474433,-0.17082486,-0.029483724,0.04382453..."


In [5]:
df['fasttext_vec']=df['document_vector_flat'].apply(lambda x:np.fromstring(x.strip('[]'),sep=','))
X=np.vstack(df['fasttext_vec'].values)
y=df['hate_speech']

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

### SMOTE

In [7]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority',random_state=42)
X_sm, y_sm = smote.fit_resample(X_train,y_train)
y_sm.value_counts()

0    13539
1    13539
Name: hate_speech, dtype: int64

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import cross_val_score

In [10]:
rand = RandomForestClassifier(n_estimators=300, max_depth=10)
scores_rand = cross_val_score(rand, X_sm,y_sm, cv = 6)
print(scores_rand.mean(), scores_rand.std())

0.8623605879311618 0.03373083551135905


In [11]:
rand.fit(X_sm,y_sm)
y_pred_rand=rand.predict(X_test)

In [31]:
print("Random forest Performance:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rand))
print(classification_report(y_test, y_pred_rand))

Random forest Performance:
Confusion Matrix:
 [[2816  570]
 [ 353  704]]
              precision    recall  f1-score   support

           0       0.89      0.83      0.86      3386
           1       0.55      0.67      0.60      1057

    accuracy                           0.79      4443
   macro avg       0.72      0.75      0.73      4443
weighted avg       0.81      0.79      0.80      4443



In [13]:
accuracy=accuracy_score(y_test,y_pred_rand)
precision=precision_score(y_test,y_pred_rand)
recall=recall_score(y_test,y_pred_rand)
f1=f1_score(y_test,y_pred_rand)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1:{f1:.2f}")

Accuracy:0.79
precision:0.55
recall:0.67
f1:0.60


### Applying hyperparameter tuning for Random Forest

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 

In [14]:
param_grid = { 
    'n_estimators': [25,50,100,150,300], 
    'max_features': ['sqrt','log2',None], 
    'max_depth': [3, 6, 9], 
    'max_leaf_nodes': [3, 6, 9], 
} 

In [17]:
random_search = RandomizedSearchCV(RandomForestClassifier(),
                                   param_grid) 
random_search.fit(X_train, y_train) 
print(random_search.best_estimator_)

RandomForestClassifier(max_depth=6, max_features=None, max_leaf_nodes=9,
                       n_estimators=150)


In [33]:
rand_random=RandomForestClassifier(max_depth=6,max_features=None, max_leaf_nodes=9,n_estimators=150)

In [34]:
rand_random.fit(X_sm,y_sm)

RandomForestClassifier(max_depth=6, max_features=None, max_leaf_nodes=9,
                       n_estimators=150)

In [35]:
y_pred_rand_r=rand_random.predict(X_test)

In [36]:
print("Random forest Performance:")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rand_r))
print(classification_report(y_test, y_pred_rand_r))

Random forest Performance:
Confusion Matrix:
 [[2722  664]
 [ 420  637]]
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      3386
           1       0.49      0.60      0.54      1057

    accuracy                           0.76      4443
   macro avg       0.68      0.70      0.69      4443
weighted avg       0.78      0.76      0.76      4443



In [37]:
accuracy=accuracy_score(y_test,y_pred_rand_r)
precision=precision_score(y_test,y_pred_rand_r)
recall=recall_score(y_test,y_pred_rand_r)
f1=f1_score(y_test,y_pred_rand_r)
print(f"Accuracy:{accuracy:.2f}")
print(f"precision:{precision:.2f}")
print(f"recall:{recall:.2f}")
print(f"f1:{f1:.2f}")

Accuracy:0.76
precision:0.49
recall:0.60
f1:0.54
